In [ ]:
import weave
# weave.use_frontend_devmode()

In [ ]:
import weave
import weave.legacy.weave
panels = weave.legacy.weave.panels

project = "tim-test-llms-test-all-final-colab-01"
entity = "timssweeney"

# Create a Weave Query to get Traces
traces = (
    weave.legacy.weave.ops
    .project(entity, project)
    .runs()
    .history().map(
        lambda row: row['langchain_trace'].dropna()
    )
    .concat()
    .dropna()
    .sort(
        lambda row: weave.legacy.weave.ops.make_list(a=row.startTime()), 
        ['asc']
    )
)

# Create a table from the query
table = panels.Table(traces)
table_state = table.config.tableState

# Add columns, set sorting, and specify panels for columns
table_state.add_column(lambda row: row.traceSummaryDict()["isSuccess"], "Success")
start_time_col_id = table_state.add_column(lambda row: row.traceSummaryDict()["startTime"].toTimestamp(), "Timestamp")
table_state.enable_sort(start_time_col_id, "desc")

markdown_panel = panels.table_state.PanelDef('string', {'mode': 'markdown'})
table_state.add_column(lambda row: row.traceSummaryDict()["formattedInput"], "Input", markdown_panel)
table_state.add_column(lambda row: row.traceSummaryDict()["formattedOutput"], "Output", markdown_panel)
table_state.add_column(lambda row: row.traceSummaryDict()["formattedChain"], "Chain", markdown_panel)
table_state.add_column(lambda row: row.traceSummaryDict()["error"], "Error")
table_state.add_column(lambda row: row.traceSummaryDict()["modelHash"], "Model ID")

# Compose into a group with detail view below
dash = panels.Group(
    items={
        "all_traces": table,
        "selected_details": lambda all_traces: panels.Card(
            title='Trace Viewer',
            subtitle='',
            content=[
                panels.CardTab(
                    name='Trace Timeline',
                    content=panels.PanelWBTraceTreeTraceViewer(all_traces.active_data())
                ),
                panels.CardTab(
                    name='Model Architecture',
                    content=panels.PanelWBTraceTreeModelViewer(all_traces.active_data())
                ),

            ]
        )
    }
)

# Publish to W&B
# weave.publish(dash, "weave_ops/prompts_in_weave1")

# View locally
dash